In [ ]:
from google.colab import drive, userdata
import requests
import pickle
import pprint
import random
import re

In [ ]:
drive.mount("/content/drive")
%cd '/content/drive/MyDrive/Colab Notebooks/Math_Graph/pickle_files'

# Get API key
wolfram = userdata.get('Wolfram_alpha')

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Math_Graph/pickle_files


In [ ]:
def read_pickle(file_name):
  with open(file_name, 'rb') as file:
    data = pickle.load(file)
  return data

In [ ]:
# Load mizar_subjects
dict_file = "mizar_subjects.pkl"
mizar_subj_list = read_pickle(dict_file)

In [ ]:
def get_wikidata_subject_info(entity_name, subject_dict):
  """Retrieves data from Wikidata API"""
  # Construct Wikidata API URL call
  url = f"https://www.wikidata.org/w/api.php?action=wbsearchentities&search={entity_name}&language=en&format=json"

  # Send a GET request to Wikidata API
  response = requests.get(url)

  # If search was successful, extract results
  if response.status_code == 200:
    search_results = response.json().get("search")
    if search_results:
      subject_dict.update({entity_name : search_results})
    else:
      print(f"No results found for {entity_name}")
  return subject_dict

In [ ]:
def load_or_create_dict(subject_dict_name, function_to_run, subj_list):
  """If pickle file not in existence, create via running appropriate API function"""

  try:
    subject_dict = read_pickle(subject_dict_name)

  except FileNotFoundError:
    print(f"{subject_dict_name} not found, creating {subject_dict_name}...")
    subject_dict = dict()
    for entity_name in subj_list:
      subject_dict = function_to_run(entity_name, subject_dict)

    with open(subject_dict_name, 'wb') as file:
      pickle.dump(subject_dict, file)

  return subject_dict

In [ ]:
def split_camel_case(text):
    # Use regex to insert a space before uppercase letters preceded by lowercase letters
    return re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', text)

In [ ]:
mizar_wikidata_dict = load_or_create_dict("mizar_wikidata_api.pkl", get_wikidata_subject_info, mizar_subj_list)

print(f"dictionary length: {len(mizar_wikidata_dict)}")
random_pairs = random.sample(list(mizar_wikidata_dict.items()), 5)

for key, value in random_pairs:
  print(f"{key}:\n{value}")

dictionary length: 96
Arrow's_impossibility_theorem:
[{'id': 'Q33481', 'title': 'Q33481', 'pageid': 36319, 'concepturi': 'http://www.wikidata.org/entity/Q33481', 'repository': 'wikidata', 'url': '//www.wikidata.org/wiki/Q33481', 'display': {'label': {'value': "Arrow's impossibility theorem", 'language': 'en'}, 'description': {'value': 'Result that no ranked-choice system is spoilerproof', 'language': 'en'}}, 'label': "Arrow's impossibility theorem", 'description': 'Result that no ranked-choice system is spoilerproof', 'match': {'type': 'label', 'language': 'en', 'text': "Arrow's impossibility theorem"}}, {'id': 'Q56659147', 'title': 'Q56659147', 'pageid': 56600106, 'concepturi': 'http://www.wikidata.org/entity/Q56659147', 'repository': 'wikidata', 'url': '//www.wikidata.org/wiki/Q56659147', 'display': {'label': {'value': "Arrow's impossibility theorem: Concise proof and infinite voters", 'language': 'en'}}, 'label': "Arrow's impossibility theorem: Concise proof and infinite voters", 'm

In [ ]:
def get_wolfram_short_answer_query_name(entity_name):
  """ Prepares the entity name for Wolfram API query: requires query to be prefixed 'What is ...'"""
  entity_name_split = entity_name.split("_")

  if len(entity_name_split) > 2:
    concatenated_subtext = '+'.join(entity_name_split[1:-1])
    concatenated = f"{entity_name_split[0]}+{concatenated_subtext}+{entity_name_split[-1]}"
  elif len(entity_name_split) == 2:
    concatenated = f"{entity_name_split[0]}+{entity_name_split[-1]}"
  else:
    concatenated = f"{entity_name_split[0]}"

  query_name = f"What+is+{concatenated}%3f"
  return query_name

def get_wolfram_alpha_subject_info(entity_name, subject_dict):
  """ Retrieves data from Wolfram short answer API'"""
  query_name = get_wolfram_short_answer_query_name(entity_name)

  # # Construct WolframAlpha API call
  url = f"http://api.wolframalpha.com/v1/result?appid={wolfram}&i={query_name}"

  # Send a GET request to WolframAlpha API
  response = requests.get(url)

  # If search was successful, extract results
  if response.status_code == 200:
    search_results = response.text
    if search_results:
      subject_dict.update({entity_name : search_results})
    else:
      print(f"No results found for {entity_name}")
  return subject_dict

In [ ]:
mizar_wolfram_alpha_dict = load_or_create_dict("mizar_short_ans_api.pkl", get_wolfram_alpha_subject_info, mizar_subj_list)

print(f"dictionary length: {len(mizar_wolfram_alpha_dict)}")
random_pairs = random.sample(list(mizar_wolfram_alpha_dict.items()), 5)

for key, value in random_pairs:
  print(f"{key}:\n{value}")

dictionary length: 65
Chinese_remainder_theorem:
Let r and s be positive integers that are relatively prime and let A and b be any two integers.  Then there is an integer N such that N is congruent both to A mod r and to b mod s.  Moreover, N is uniquely determined modulo r times s
Quotient_ring:
A quotient ring (also called a residue‐class ring) is a ring that is the quotient of a ring A and one of its ideals a, denoted A/a.  For example, when the ring A is Z (the integers) and the ideal is 6Z (multiples of 6), the quotient ring is Z_6 equals Z/6Z.  In general, a quotient ring is a set of equivalence classes where [x] equals [y] iff x-y∈a.  The quotient ring is an integral domain iff the ideal a is prime.  A stronger condition occurs when the quotient ring is a field, which corresponds to when the ideal a is maximal
Jordan_curve_theorem:
If J is a simple closed curve in R^2, then the Jordan curve theorem, also called the Jordan‐Brouwer theorem states that R^2-J has two components (an 

In [ ]:
# Repeat for additional scraped topics from wolfram math world

In [ ]:
# Load topics
dict_file = "topics.pkl"
topics = read_pickle(dict_file)

# Load sub_topics
dict_file = "sub_topics.pkl"
sub_topics = read_pickle(dict_file)

# Load alg_2
dict_file = "alg_2.pkl"
alg_2 = read_pickle(dict_file)

# Load alg_3
dict_file = "alg_3.pkl"
alg_3 = read_pickle(dict_file)

In [ ]:
scraped_subj_list = []
for key, value in topics.items():
  scraped_subj_list.append(key)
  scraped_subj_list.extend(value)

for key, value in sub_topics.items():
  key = split_camel_case(key)
  scraped_subj_list.append(key)
  scraped_subj_list.extend(value)

for key, value in alg_2.items():
  scraped_subj_list.append(key)
  scraped_subj_list.extend(value)

for key, value in alg_3.items():
  scraped_subj_list.append(key)
  scraped_subj_list.extend(value)

scraped_subj_list = list(set(scraped_subj_list))
scraped_subj_list = [x.replace(" ", "_") for x in scraped_subj_list]

In [ ]:
scraped_wikidata_dict = load_or_create_dict("wolfram_wikidata_api.pkl", get_wikidata_subject_info, scraped_subj_list)

print(f"dictionary length: {len(scraped_wikidata_dict)}")
random_pairs = random.sample(list(scraped_wikidata_dict.items()), 5)

for key, value in random_pairs:
  print(f"{key}:\n{value}")

dictionary length: 3227
Integral_Element:
[{'id': 'Q1493740', 'title': 'Q1493740', 'pageid': 1431393, 'concepturi': 'http://www.wikidata.org/entity/Q1493740', 'repository': 'wikidata', 'url': '//www.wikidata.org/wiki/Q1493740', 'display': {'label': {'value': 'integral element', 'language': 'en'}, 'description': {'value': 'Mathematical element', 'language': 'en'}}, 'label': 'integral element', 'description': 'Mathematical element', 'match': {'type': 'label', 'language': 'en', 'text': 'integral element'}}]
Integrally_Closed:
[{'id': 'Q6042740', 'title': 'Q6042740', 'pageid': 5827375, 'concepturi': 'http://www.wikidata.org/entity/Q6042740', 'repository': 'wikidata', 'url': '//www.wikidata.org/wiki/Q6042740', 'display': {'label': {'value': 'integrally closed', 'language': 'en'}, 'description': {'value': 'Wikimedia disambiguation page', 'language': 'en'}}, 'label': 'integrally closed', 'description': 'Wikimedia disambiguation page', 'match': {'type': 'label', 'language': 'en', 'text': 'inte

In [ ]:
scraped_wolfram_alpha_dict = load_or_create_dict("wolfram_short_ans_api.pkl", get_wolfram_alpha_subject_info, scraped_subj_list)

print(f"dictionary length: {len(scraped_wolfram_alpha_dict)}")
random_pairs = random.sample(list(scraped_wolfram_alpha_dict.items()), 5)

for key, value in random_pairs:
  print(f"{key}:\n{value}")

dictionary length: 5026
Lochs'_Theorem:
For a real number x∈(0,1), let m be the number of terms in the convergent to a regular continued fraction that are required to represent n decimal places of x.  Then for almost all x, lim_n->∞m/n equals 6ln2ln10/π^2 equals 0.97027014…(OEIS A086819; Lochs 1964).  This number is sometimes known as Lochs’ constant.  Therefore, the regular continued fraction is only slightly more efficient at representing real numbers than is the decimal expansion.  The set of x for which this statement does not hold is of measure 0
Cube_Dovetailing_Problem:
Given the above figure (without looking at the figure below!), determine how to disengage the two slotted cube halves without cutting, breaking, or distorting.  One possible solution is shown above; the slots are not perpendicular to one another but instead consist of parallel slotted tracks.  Other solutions are also possible.  For example, another construction involves two circular arcs sharing the same center
